# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, json
import gmaps
# import json
import os

# Import API key
from config import key

In [2]:
# Load city data from WeatherPy
city_data = "../WeatherPy/Output/city_data.csv"
city_data_df = pd.read_csv(city_data, index_col=[0], encoding="ISO-8859-1")
city_data_df.head()
city_data_df.dtypes

City_Name       object
City_Lat       float64
City_Long      float64
Temperature    float64
Humidity         int64
Cloudiness       int64
Wind_Speed     float64
Country         object
Date            object
dtype: object

In [3]:
# Rename columns and create new dataframe with dropped NA values 
city_data_df = city_data_df.rename(columns={'City_Name': 'City', 'City_Lat': 'Latitude', 
                                            'City_Long': 'Longitude', 'Wind_Speed': 'Wind Speed'})

city_loc = city_data_df.dropna(axis = 0, how ='any')
# city_data_cleandf = city_data_dropna[(city_data_dropna != 0).all(1)]

# comparing sizes of data frames
print("Old data frame length:", len(city_data_df), "\nNew data frame length:", 
       len(city_loc), "\nNumber of rows with at least 1 NA value: ",
       (len(city_data_df)-len(city_loc)))
city_loc.to_csv("./output_data/city_loc.csv")

Old data frame length: 645 
New data frame length: 639 
Number of rows with at least 1 NA value:  6


### Humidity Heatmap

In [4]:
# Configure gmaps
gmaps.configure(api_key=key)

# Set Locations for the heatmap
locations = city_loc[["Latitude", "Longitude"]]

humidity = city_loc["Humidity"].astype(float)

In [5]:
# Plot humidity heatmap
fig = gmaps.figure()

# Create humidity heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=100, point_radius=3)
# Add layer and display fig
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Ideal Weather Conditions

In [6]:
# Create new dataframe based on indicated parameters
ideal_cities = city_loc[(city_loc['Temperature']>80) & (city_loc['Temperature']<90) 
                   & (city_loc['Humidity']>40) & (city_loc['Humidity']<70) 
                   & (city_loc['Wind Speed']<10) & (city_loc['Cloudiness']<20)]
ideal_cities.head()
# len(ideal_cities)    #---- used to check number of cities that meet criteria; uncomment to enable 

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
13,Rome,34.2570,-85.1647,84.45,42,1,1.99,US,2021-06-17 17:57:41
79,Hamilton,39.1834,-84.5333,83.10,44,19,1.99,US,2021-06-17 18:00:10
99,Russell,32.3502,-85.1999,88.86,42,0,4.50,US,2021-06-17 18:00:06
260,Prainha,-1.8000,-53.4800,87.44,55,13,4.03,BR,2021-06-17 18:01:38
271,Havelock,34.8791,-76.9013,82.76,61,0,4.00,US,2021-06-17 18:01:43


### Find Nearest Hotel

In [9]:
# Create new dataframe of city geo data
hotels_df = ideal_cities.loc[:, ['City', 'Country', 'Latitude', 'Longitude', 'Date']].copy()

# Add column to hotels dataframe
hotels_df['Hotel Name'] = ""

In [11]:
# Parameters dictionary for Nearby search requests
params = {
#     'location': 'city_geo',
    'types': 'lodging', # keyword
    'radius': 5000, 
    'key': key
}
# Nearby search base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through pandas dataframe
for i, row in hotels_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params['location'] = f'{lat},{long}'

    try:
        # Construct url and make API request
        response = requests.get(base_url, params)
        json_results = response.json()
        hotel_name = json_results['results'][0]['name']
        hotels_df['Hotel Name'][i] = hotel_name      

    except:
        print('Missing field for city of {}...skipping'.format(row['City']))
        print('-'*26)

hotels_df.to_csv('./output_data/hotels_df.csv')
hotels_df.head()

C:\Users\jchan\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Country,Latitude,Longitude,Date,Hotel Name
13,Rome,US,34.2570,-85.1647,2021-06-17 17:57:41,La Quinta Inn & Suites by Wyndham Rome
79,Hamilton,US,39.1834,-84.5333,2021-06-17 18:00:10,Six Acres Bed & Breakfast
99,Russell,US,32.3502,-85.1999,2021-06-17 18:00:06,Macs
260,Prainha,BR,-1.8000,-53.4800,2021-06-17 18:01:38,Casa Rocha
271,Havelock,US,34.8791,-76.9013,2021-06-17 18:01:43,Sherwood Motel
